# Assignment: Segmenting and Clustering Neighborhoods in Toronto 

#### Kaylene Robinson


For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1.) Start by creating a new Notebook for this assignment.

2.) Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

### Import Libraries

In [84]:
# Library data analysis
import pandas as pd
# Library to handle vectorized data
import numpy as np

# Library for web scraping
import requests
from bs4 import BeautifulSoup

print('Libraries Imported!')

Libraries Imported!


## Part 1

#### Objective: Scrape the Wikipedia page data, clean data, and use .shape method to create dataframe.

In [85]:
#Import data from Wikipedia
wiki_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_text= requests.get(wiki_link).text

#Use BeautifulSoup to parse data
Canada_data = BeautifulSoup(wiki_text, 'lxml')

In [86]:
#Extract raw table data inside Wikipedia webpage
table = Canada_data.find('table')

def table_cell(i):
    cells = i.find_all('td')
    row = []
    
    for cell in cells:
        if cell.a:            
            if (cell.a.text):
                row.append(cell.a.text)
                continue
        row.append(cell.string.strip())
        
    return row

def table_row():    
    data = []  
    
    for tr in table.find_all('tr'):
        row = table_cell(tr)
        if len(row) != 3:
            continue
        data.append(row)        
    
    return data

In [87]:
data = table_row()
columns = ['Postcode', 'Borough', 'Neighbourhood']
df_toronto = pd.DataFrame(data, columns=columns)
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [88]:
#Ignore "Not Assigned" boroughs
df2 = df_toronto[df.Borough != 'Not assigned']
df2 = df1.sort_values(by=['Postcode','Borough'])

df2.reset_index(inplace=True)
df2.drop('index',axis=1,inplace=True)
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Highland Creek
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Port Union


In [89]:
#Combine neighbourhoods that share the same postcode
df_postcodes = df2['Postcode']
df_postcodes.drop_duplicates(inplace=True)
df3 = pd.DataFrame(df_postcodes)
df3['Borough'] = '';
df3['Neighbourhood'] = '';


df3.reset_index(inplace=True)
df3.drop('index', axis=1, inplace=True)
df2.reset_index(inplace=True)
df2.drop('index', axis=1, inplace=True)

for i in df3.index:
    for j in df1.index:
        if df3.iloc[i, 0] == df2.iloc[j, 0]:
            df3.iloc[i, 1] = df2.iloc[j, 1]
            df3.iloc[i, 2] = df3.iloc[i, 2] + ',' + df2.iloc[j, 2]
            
for i in df3.index:
    s = df3.iloc[i, 2]
    if s[0] == ',':
        s =s [1:]
    df3.iloc[i,2 ] = s
    
df3.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [90]:
#Checking dataframe shape
df3.shape

(103, 3)

## Part 2

#### Objective: Utilize the Foursquare location data by obtaining the latitude and the longitude coordinates of each neighborhood, and use Geocoder package or the csv file to create dataframe.

In [91]:
#Import additional libraries
!pip install geocoder
!pip install Nominatim
!pip install geopy

In [92]:
#Reading the file to coord dataframe
df_geopy = df3
df_geopy['Latitude'] = '0';
df_geopy['Longitude'] = '0';

coord = pd.read_csv('https://cocl.us/Geospatial_data')

In [93]:
#Create dataframe that contains coordinates with borough names
for i in df_geopy.index:
    for j in coord.index:
        if df_geopy.iloc[i, 0] == coord.iloc[j, 0]:
            df_geopy.iloc[i, 3] = coord.iloc[j, 1]
            df_geopy.iloc[i, 4] = coord.iloc[j, 2]

#check the results            
df_geopy.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


## Part 3

#### Objective: Explore and cluster the neighborhoods in Toronto. Work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data.

###### Part 3 (a.)

In [118]:
# Select the neighbourhoods that contain word " Downtown Toronto"
DTT = df_geopy[df_geopy['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
DTT.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.6796,-79.3775
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.668,-79.3677
2,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832
3,M5A,Downtown Toronto,Harbourfront,43.6543,-79.3606
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.6572,-79.3789


In [119]:
#Find coordinates for Toronto
address = 'Toronto, Canada'

from  geopy.geocoders import Nominatim
geolocator = Nominatim()

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Toronto Geographic Coordinates: {}, {}.'.format(latitude, longitude))

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


Toronto Geographic Coordinates: 43.653963, -79.387207.


In [120]:
# Install additional libraries
!pip install folium
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [121]:
#Create a folium map of Downtown Toronto
map_DTT = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for latitude, longitude, label in zip(DTT['Latitude'], DTT['Longitude'], DTT['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_DTT

##### Part 3 (b.) Utilize the Foursquare API to get top 100 venues in Downtown Toronto

In [123]:
#Enter Foursquare credintials
CLIENT_ID = 'MLQC5UDW1DG2UCP4KV5FB1PFAQHYVPYWKBRTLHSAAKQMLXR1'
CLIENT_SECRET = 'NQPDSFHZHZ1TOONTR2Z2PMTLQYQV202MJECGQN4KAVJ4GT2D' 
VERSION = '20191117'

In [162]:
#Borrow the function from the lab to get Top 100 venues in Downtown Toronto within a radius of 500m
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # Create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # Make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # Return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [176]:
# Set limit of number of venues returned by Foursquare API
LIMIT = 100 

DTT_venues = getNearbyVenues(names=DTT['Neighbourhood'],
                                   latitudes=DTT['Latitude'],
                                   longitudes=DTT['Longitude']
                                  )
DTT_venues

Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"Cabbagetown,St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner
5,"Cabbagetown,St. James Town",43.667967,-79.367675,F'Amelia,43.667536,-79.368613,Italian Restaurant
6,"Cabbagetown,St. James Town",43.667967,-79.367675,Kingyo Toronto,43.665895,-79.368415,Japanese Restaurant
7,"Cabbagetown,St. James Town",43.667967,-79.367675,Cabbagetown Brew,43.666923,-79.369289,Café
8,"Cabbagetown,St. James Town",43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant
9,"Cabbagetown,St. James Town",43.667967,-79.367675,Murgatroid,43.667381,-79.369311,Restaurant


In [177]:
# Check the size of venues dataframe
DTT_venues.shape

(1287, 7)

In [178]:
# Check how many unique categories of venues are there
print('There are {} unique categories.'.format(len(DTT_venues['Venue Category'].unique())))

There are 201 unique categories.


###### Part 3 (c.) Analyze each neighborhood in Downtown Toronto

In [224]:
# One hot encoding
dt_onehot = pd.get_dummies(DTT_venues[['Venue Category']], prefix="", prefix_sep="")

# Move neighborhood column to the first column
fixed_columns = [dt_onehot.columns[-1]] + list(dt_onehot.columns[:-1])
dt_onehot = dt_onehot[fixed_columns]

dt_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [218]:
dt_onehot.shape

(1287, 201)

In [226]:
# Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category:
# Add neighborhood column back to dataframe
dt_onehot['Neighborhood'] = DTT_venues
dt_onehot.groupby('Neighborhood')
dt_grouped = dt_onehot.groupby('Neighborhood').mean().reset_index()
dt_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.030000,...,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.012500,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.012500,...,0.000000,0.00,0.00,0.00,0.012500,0.000000,0.000000,0.012500,0.000000,0.000000
5,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.041667,0.000000,0.052083,0.010417,0.000000,0.000000
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Church and Wellesley,0.011236,0.011236,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011236,...,0.011236,0.00,0.00,0.00,0.000000,0.011236,0.011236,0.000000,0.011236,0.011236
8,"Commerce Court,Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,...,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
9,"Design Exchange,Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,...,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000


In [228]:
# Print out each neighborhood along with the top 5 most common venues in it

top_venues = 5

for hood in dt_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dt_grouped[dt_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(top_venues))
    print('\n')

----Adelaide,King,Richmond----
            venue  freq
0     Coffee Shop  0.07
1            Café  0.05
2      Steakhouse  0.04
3             Bar  0.04
4  Cosmetics Shop  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1              Bakery  0.05
2            Beer Bar  0.04
3        Cocktail Bar  0.04
4  Seafood Restaurant  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.20
1  Airport Terminal  0.13
2    Airport Lounge  0.13
3   Harbor / Marina  0.07
4       Coffee Shop  0.07


----Cabbagetown,St. James Town----
                venue  freq
0          Restaurant  0.07
1         Coffee Shop  0.07
2       Grocery Store  0.05
3  Italian Restaurant  0.05
4              Bakery  0.05


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.14
1                Café  0.05
2      Ice Cream Shop  0.05
3  Italian Restaurant 

In [238]:
# Sort the venues in descending order:
def return_most_common_venues(row, top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:top_venues]

In [239]:
# Create the new dataframe and display the top 10 venues for each neighborhood:

top_venues = 10

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neigh_venues_sorted = pd.DataFrame(columns=columns)
neigh_venues_sorted['Neighborhood'] = dt_grouped['Neighborhood']

for ind in np.arange(dt_grouped.shape[0]):
    neigh_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], top_venues)

neigh_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Steakhouse,Breakfast Spot,Restaurant,Bakery,Asian Restaurant,Thai Restaurant,Sushi Restaurant
1,Berczy Park,Coffee Shop,Bakery,Seafood Restaurant,Beer Bar,Farmers Market,Cocktail Bar,Cheese Shop,Steakhouse,Café,Hotel
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Boutique,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Harbor / Marina
3,"Cabbagetown,St. James Town",Restaurant,Coffee Shop,Grocery Store,Pub,Café,Pizza Place,Bakery,Park,Italian Restaurant,Playground
4,Central Bay Street,Coffee Shop,Café,Ice Cream Shop,Italian Restaurant,Sandwich Place,Burger Joint,Bubble Tea Shop,Bar,Bakery,Chinese Restaurant


##### Part 3 (d.) Run k-means to cluster the neighborhood into 5 clusters.

In [242]:
# Import additional library
from sklearn.cluster import KMeans

# Set number of clusters
kclusters = 4

dt_grouped_clustering = dt_grouped.drop('Neighborhood', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 2, 3, 3, 3, 0, 3, 3, 3], dtype=int32)

In [ ]:
pd.concat(neigh_venues_sorted.insert(0, 'Label', kmeans.labels_))

dt_merged = dt_merged.join(neigh_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
dt_merged.head()

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_merged['Latitude'], dt_merged['Longitude'], dt_merged['Neighbourhood'], dt_merged['Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=9,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters